kNN implementation with Spark

Goals:
* Improve understanding of the kNN algorithm by implementing it
* Develop pyspark skills using **core** transformations, and actions
* Develop numPy and vectorization skills by using numPy arrays for implementation.
* Learn how to write scalable pySpark code by (possibly modifying and) running pySpark code on AWS

## Data

The dataset is the widely-used “20 newsgroups” dataset. A newsgroup post is like an old-school blog post, and this dataset has 19,997 such posts from 20 different categories, according to where the blog post was made.

The 20 categories are listed in the file `news_categories.txt`.

For each document, the category name can be extracted from the id of the document. For example,
* the document with the id `20_newsgroups/comp.graphics/37261` is from the `comp.graphics` category,
* the document with the id `20_newsgroups/sci.med/59082` is from the `sci.med` category.

The data file has one line per document of text. It can be accessed at:

`s3://risamyersbucket/Lab/text/20_news_same_line.txt`

# Install Java, Spark, and Findspark
This installs Apache Spark, Java, and [Findspark](https://github.com/minrk/findspark), a library that makes it easy for Python to find Spark.


In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install -q pyspark > /dev/null

## Set Environment Variables

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"
os.environ["PYTHONHASHSEED"] = "0"


In [ ]:
### Start a pySpark Session
import findspark
findspark.init()
import pyspark
from pyspark import SparkContext, SparkConf

sc = SparkContext.getOrCreate()


In [ ]:
# Load useful modules
import re
import numpy as np

## Compute "bag of words" for each document

For task 1, we want to extract "bag of words" features for documents.
`
[('mostcommonword', 0),
 ('nextmostcommonword', 1),
 ...]
`

**NOTE**: There aren’t 20,000 unique words in the small dataset (`20_news_same_line_random_sample.txt`). Use only the top 100 words when working with this file.

**Provided code to create the reference dictionary of words.**

Run the code cells below to create the `refDict` RDD.

In [ ]:
# set the number of dictionary words
# 100 for the small dataset
# 20,000 for the large dataset
numWords = 100

In [ ]:
# load up the dataset
# "20_news_same_line_random_sample.txt" for small dataset
corpus = sc.textFile ("20_news_same_line_random_sampleV3.txt")

# each entry in validLines will be a line from the text file
validLines = corpus.filter(lambda x : 'id=' in x)

# now we transform it into a bunch of (docID, text) pairs
keyAndText = validLines.map(lambda x : (x[x.index('id="') + 4 : x.index('" url=')], x[x.index('"> ') + 3:x.index(' </doc>')]))

# now we split the text in each (docID, text) pair into a list of words
# after this, we have a data set with (docID, ["word1", "word2", "word3", ...])
# we have a bit of fancy regular expression stuff here to make sure that we do not
# die on some of the documents
regex = re.compile('[^a-zA-Z]')
keyAndListOfWords = keyAndText.map(lambda x : (str(x[0]), regex.sub(' ', x[1]).lower().split()))

# now get the top 20,000 words... first change (docID, ["word1", "word2", "word3", ...])
# to ("word1", 1) ("word2", 1)...
allWords = keyAndListOfWords.flatMap(lambda x: ((j, 1) for j in x[1]))

# now, count all of the words, giving us ("word1", 1433), ("word2", 3423423), etc.
allCounts = allWords.reduceByKey (lambda a, b: a + b)

# and get the top numWords (100 for small dataset, 20K for large dataset) frequent words in a local array
topWords = allCounts.top (numWords, lambda x : x[1])

# and we'll create an RDD that has a bunch of (word, rank) pairs
# start by creating an RDD that has the number 0 up to numWords (100 for small dataset, 20K for large dataset)
# numWords is the number of words that will be in our dictionary
numWordsRDD = sc.parallelize(range(numWords))

# now, we transform (0), (1), (2), ... to ("mostcommonword", 0) ("nextmostcommon", 1), ...
# the number will be the spot in the dictionary used to tell us where the word is located
refDict = numWordsRDD.map(lambda x:(topWords[x][0],x))

In [ ]:
refDict.take(10)

[('the', 0),
 ('to', 1),
 ('a', 2),
 ('of', 3),
 ('and', 4),
 ('i', 5),
 ('in', 6),
 ('is', 7),
 ('that', 8),
 ('it', 9)]

Create a new RDD, named `bag_of_words`. Each element of this RDD corresponds to one document, and is a key-value pair. Specifically, the key is the document identifier `id` (e.g. `20_newsgroups/comp.graphics/37261`) and the value is a `numpy` array with `numWords` (100 for the small dataset, 20K for the large dataset) entries, where the *i*th entry in the array is the number of times that the *i*th word in the `refDict` (created in the first part) appears in the document. This array corresponds to the "bag of words" features for each document.

Since each array is going to be huge, with a lot of zeros, print out only the non-zero entries in the array (that is, for an array `a`, print out `a[a.nonzero()]`.

In [ ]:
# create first RDD to organize from previous RDD in format (word, docID)
RDD1= keyAndListOfWords.flatMap(lambda x: ((i, x[0])for i in x[1]))

# modify so that refDict is joined with new RDD created, in format (word, (docID, num (rank of word) in refdict)
RDD2= RDD1.join(refDict)

# # now, remove the characters from new RDD
RDD3= RDD2.map(lambda x: ((x[1]))) # (docID, num in refdict)


In [ ]:
# make an array of 0s for length 100
zeros= np.zeros(100)

# Spark4Actions pg 9 - code all given in slide
# first, combine values in chunk against eachother if same
def seq_func(accumulator, element):
  #sum the values of the same element, counter
  accumulator[element]=accumulator[element]+1
  return accumulator

# next, combine values across chunk against eachother if same
def comb_func(accumulator1, accumulator2):
  # sum across the partitions
  sum= accumulator1+accumulator2
  return sum

In [ ]:
bag_of_words = RDD3.aggregateByKey (zeros,seq_func, comb_func) #pass 3 things

Print results by running the code cells below:

In [ ]:
arr1_1 = np.array(bag_of_words.filter(lambda x: "20_newsgroups/soc.religion.christian/21626" in x[0]).collect()[0][1])
arr1_1[arr1_1.nonzero()]

array([ 7.,  2.,  4., 10.,  4.,  5.,  1.,  6.,  7.,  8.,  1.,  1.,  1.,
        3.,  2.,  1.,  1.,  1.,  1.,  2.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  3.,  1.,  1.,  1.,  1.,  1.,  1.,  4.,  1.,  1.,
        2.,  1.,  2.,  1.,  1.,  1.])

In [ ]:
arr1_2 = np.array(bag_of_words.filter(lambda x: "20_newsgroups/talk.politics.misc/179019" in x[0]).collect()[0][1])
arr1_2[arr1_2.nonzero()]

array([ 7., 23., 17.,  5.,  6.,  5., 14., 10.,  3., 20., 15., 11.,  4.,
        1.,  1.,  4.,  4.,  8.,  3.,  4.,  2.,  1.,  2.,  3., 10.,  3.,
        1.,  1.,  1.,  1.,  1.,  2.,  1.,  1.,  1.,  1.,  2.,  2.,  2.,
        3.,  2.,  1.,  1.,  5.,  2.,  1.,  1.,  1.,  8.,  3.,  1.,  1.,
        1.,  2.,  1.,  2.,  1.,  1.,  2.,  1.,  2.,  2.])

In [ ]:
arr1_3 = np.array(bag_of_words.filter(lambda x: "20_newsgroups/rec.autos/103167" in x[0]).collect()[0][1])
arr1_3[arr1_3.nonzero()]

array([9., 1., 2., 3., 2., 1., 1., 1., 1., 1., 1., 1., 1., 2., 1., 1., 1.,
       1., 3., 1., 1., 1., 1., 1., 1., 1., 2., 1., 2., 1.])


## Compute the TF-IDF value for each document

It is often difficult to classify documents accurately using raw count vectors (bag of words). Thus, the next task is to write some more Spark RDD transformations and actions that convert each of the count vectors to TF-IDF vectors (numpy arrays).

Create an RDD of key-value pairs, named `tfidf`, where the keys are document identifiers, and the values are the TF-IDF vector (represented as a numpy array) for that document. Again, we are only interested in the **top** `numWords` (100 for small dataset, 20K for large dataset) most common words.  

The *i*th entry in a TF-IDF vector corresponds to the *i*th word in the **top** `numWords` most common words dictionary `refDict`. Then, the *i*th entry in a TF-IDF vector for document $d$ in a corpus with $D$ documents is computed as:

$$ TF(i, d) \times IDF(i, D) $$

Where $TF(i, d)$ is:

$$ \frac {\textrm{# of occurences of word $i$ in $d$}} {\textrm{Total # of words in $d$}} $$

Note that the “Total number of words” is not the number of distinct words. The “total number of words”
in “Today is a great day today” is six.

And the $IDF(i,D)$ is:

$$ \log \frac {\textrm{# of documents in corpus D}} {\textrm{# of documents having word $i$}} $$

Use the numPy [log](https://https://numpy.org/doc/stable/reference/generated/numpy.log.html) function (natural log).

In [ ]:
# disable exponential notation
np.set_printoptions(suppress=True)

numdoc = int(bag_of_words.count()) # first find # of documents in corpus D

# create (rank, (id1, id2, ...)) pair
rankAndid = RDD2.map(lambda x: (x[1][1], x[1][0]))
rankAndidcnt = rankAndid.groupByKey()
# create (rank, # of documents having word) pair
rankAndocc = rankAndidcnt.map(lambda x: (x[0], len(set(x[1]))))
# sort based on rank
rankAndoccSorted = rankAndocc.sortBy(lambda x: x[0])
# create idf as list using idf formula above
idf = rankAndoccSorted.map(lambda x: (np.log(numdoc/x[1]))).collect() # in this case, # of documents in corpus D is numdoc
# create tf_idf RDD
# for each pair in bag_of_words, calculate TF for each word, then use matrix multiplication with IDF
tf_idf = bag_of_words.map(lambda x: (x[0], np.multiply((x[1]/x[1].sum()),idf)))

In [ ]:
arr2_1 = tf_idf.filter(lambda x: x[0]=='20_newsgroups/soc.religion.christian/21626').values()
arr2_1 = arr2_1.collect()[0]
arr2_1[arr2_1.nonzero()]

array([0.00498593, 0.0027183 , 0.00558006, 0.02140703, 0.0079482 ,
       0.00949249, 0.00155193, 0.0148116 , 0.02311692, 0.02676792,
       0.00481739, 0.00495321, 0.01969   , 0.01028608, 0.00595945,
       0.00514304, 0.00582799, 0.00662376, 0.01320715, 0.00005222,
       0.00733535, 0.0084834 , 0.00157629, 0.00929397, 0.01098612,
       0.01003302, 0.02679371, 0.00992003, 0.00911096, 0.01032123,
       0.01264814, 0.01089281, 0.01232144, 0.04271363, 0.0116649 ,
       0.01250162, 0.02627947, 0.02389596, 0.04994971, 0.01482438,
       0.02510224, 0.01420196])

In [ ]:
arr2_2 = tf_idf.filter(lambda x: x[0]=='20_newsgroups/talk.politics.misc/179019').values()
arr2_2 = arr2_2.collect()[0]
arr2_2[arr2_2.nonzero()]

array([0.00201045, 0.01260504, 0.00956261, 0.00431593, 0.00480738,
       0.00312889, 0.01393565, 0.0133162 , 0.00404757, 0.04404556,
       0.01962468, 0.0256024 , 0.00869349, 0.0019425 , 0.00798906,
       0.01058602, 0.01820369, 0.00622142, 0.00961201, 0.00414761,
       0.00319188, 0.00469999, 0.00578676, 0.02670869, 0.01099002,
       0.00002106, 0.00024235, 0.00134919, 0.00684145, 0.0006356 ,
       0.00374757, 0.00394377, 0.00357064, 0.00885978, 0.00755923,
       0.00486722, 0.01114733, 0.00710066, 0.00510006, 0.00360131,
       0.019552  , 0.00627329, 0.00815623, 0.00435498, 0.00496832,
       0.03720002, 0.01512293, 0.00467671, 0.00482682, 0.00491113,
       0.01107961, 0.00550675, 0.01221835, 0.0055899 , 0.00597757,
       0.01145319, 0.00588614, 0.01289577, 0.01345513])

In [ ]:
arr2_3 = tf_idf.filter(lambda x: x[0]=='20_newsgroups/rec.autos/103167').values()
arr2_3 = arr2_3.collect()[0]
arr2_3[arr2_3.nonzero()]

array([0.01363931, 0.00289181, 0.00845553, 0.01211807, 0.00660395,
       0.00525234, 0.00702642, 0.00690343, 0.01024976, 0.01396454,
       0.01094264, 0.01267968, 0.02188527, 0.01405016, 0.00011111,
       0.05636151, 0.00127881, 0.00711913, 0.00335381, 0.01873364,
       0.02110644, 0.05481859, 0.03012542, 0.11523618, 0.03526483,
       0.06288773, 0.03402288])

## build a kNN classifier

This function will take as input a text string (`test_doc`) and a number *k*, and then output the name of one of the 20 newsgroups. This name is the news group category that the classifier thinks that the text string is “closest” to. It is computed using the classical kNN algorithm.

In [ ]:
# k is the number of neighbors to consider
# test_doc is the text to compare
def predictLabel (k, test_doc):
    # your code here
    regex = re.compile('[^a-zA-Z]')
    words = sc.parallelize(regex.sub(' ', test_doc).lower().split()) # first split the doc into list of  # https://docs.python.org/3/library/stdtypes.html
    allWordsCnt = words.map(lambda x: (x,1)).reduceByKey(lambda a,b:a+b) # count all of the words to pair (word, cnt)
    allWordsCntandRank = allWordsCnt.join(refDict) # join in format of (word, (cnt, rank))

    rankandCnt = allWordsCntandRank.map(lambda x: (x[1][1], x[1][0])) # get (rank, cnt) pair

    # create a count vector for this test doc, so it is easier to calculate TF using matrix multiplication
    countvec = np.zeros(100)
    for pair in rankandCnt.collect():
      countvec[pair[0]] = pair[1]

    tf_idf_test = np.multiply(countvec/countvec.sum(), idf) # calculate TF-IDF for this doc

    # calculate l-2 distance
    l2_dist = tf_idf.map(lambda x: (x[0], np.linalg.norm(x[1]-tf_idf_test))) # for each doc id, the l2 distance with test doc

    knn_candidate = l2_dist.top(k, key = lambda x: -x[1]) # find top k neighbors with min distance

    # need to find the most frequent category in those top k
    groupnames = [(n[0].split('/')[1], n[1]) for n in knn_candidate] # we only need the category name and dist in this list
    category_counts = {}
    # summarize the occurance of each category in a dictionary
    for cat, dist in groupnames:
      if cat in category_counts:
        category_counts[cat] += 1 #https://docs.python.org/3/reference/simple_stmts.html
      else:
        category_counts[cat] = 1

    # find the most frequent category
    max_count = max(category_counts.values())
    most_common_categories = [category for category, count in category_counts.items() if count == max_count]

    if len(most_common_categories) == 1: # if no tie happens, then just return the most frequent one
      return most_common_categories[0]
    else: # if tie happens
      min_dist = float('inf')
      min_category = None
      for category, dist in groupnames: # find the category among the ties with shortest distance
          if category in most_common_categories and dist < min_dist:
            min_dist = dist
            min_category = category
      return min_category


#### Test cases

In [ ]:
print(predictLabel (10, 'Graphics are pictures and movies created using computers – usually referring to image data created by a computer specifically with help from specialized graphical hardware and software. It is a vast and recent area in computer science. The phrase was coined by computer graphics researchers Verne Hudson and William Fetter of Boeing in 1960. It is often abbreviated as CG, though sometimes erroneously referred to as CGI. Important topics in computer graphics include user interface design, sprite graphics, vector graphics, 3D modeling, shaders, GPU design, implicit surface visualization with ray tracing, and computer vision, among others. The overall methodology depends heavily on the underlying sciences of geometry, optics, and physics. Computer graphics is responsible for displaying art and image data effectively and meaningfully to the user, and processing image data received from the physical world. The interaction and understanding of computers and interpretation of data has been made easier because of computer graphics. Computer graphic development has had a significant impact on many types of media and has revolutionized animation, movies, advertising, video games, and graphic design generally.'))


sci.space


In [ ]:
print(predictLabel (10, 'A deity is a concept conceived in diverse ways in various cultures, typically as a natural or supernatural being considered divine or sacred. Monotheistic religions accept only one Deity (predominantly referred to as God), polytheistic religions accept and worship multiple deities, henotheistic religions accept one supreme deity without denying other deities considering them as equivalent aspects of the same divine principle, while several non-theistic religions deny any supreme eternal creator deity but accept a pantheon of deities which live, die and are reborn just like any other being. A male deity is a god, while a female deity is a goddess. The Oxford reference defines deity as a god or goddess (in a polytheistic religion), or anything revered as divine. C. Scott Littleton defines a deity as a being with powers greater than those of ordinary humans, but who interacts with humans, positively or negatively, in ways that carry humans to new levels of consciousness beyond the grounded preoccupations of ordinary life.'))


soc.religion.christian


In [ ]:
print(predictLabel (10, 'Egypt, officially the Arab Republic of Egypt, is a transcontinental country spanning the northeast corner of Africa and southwest corner of Asia by a land bridge formed by the Sinai Peninsula. Egypt is a Mediterranean country bordered by the Gaza Strip and Israel to the northeast, the Gulf of Aqaba to the east, the Red Sea to the east and south, Sudan to the south, and Libya to the west. Across the Gulf of Aqaba lies Jordan, and across from the Sinai Peninsula lies Saudi Arabia, although Jordan and Saudi Arabia do not share a land border with Egypt. It is the worlds only contiguous Eurafrasian nation. Egypt has among the longest histories of any modern country, emerging as one of the worlds first nation states in the tenth millennium BC. Considered a cradle of civilisation, Ancient Egypt experienced some of the earliest developments of writing, agriculture, urbanisation, organised religion and central government. Iconic monuments such as the Giza Necropolis and its Great Sphinx, as well the ruins of Memphis, Thebes, Karnak, and the Valley of the Kings, reflect this legacy and remain a significant focus of archaeological study and popular interest worldwide. Egypts rich cultural heritage is an integral part of its national identity, which has endured, and at times assimilated, various foreign influences, including Greek, Persian, Roman, Arab, Ottoman, and European. One of the earliest centers of Christianity, Egypt was Islamised in the seventh century and remains a predominantly Muslim country, albeit with a significant Christian minority.'))


sci.space


In [ ]:
print(predictLabel (10, 'The term atheism originated from the Greek atheos, meaning without god(s), used as a pejorative term applied to those thought to reject the gods worshiped by the larger society. With the spread of freethought, skeptical inquiry, and subsequent increase in criticism of religion, application of the term narrowed in scope. The first individuals to identify themselves using the word atheist lived in the 18th century during the Age of Enlightenment. The French Revolution, noted for its unprecedented atheism, witnessed the first major political movement in history to advocate for the supremacy of human reason. Arguments for atheism range from the philosophical to social and historical approaches. Rationales for not believing in deities include arguments that there is a lack of empirical evidence; the problem of evil; the argument from inconsistent revelations; the rejection of concepts that cannot be falsified; and the argument from nonbelief. Although some atheists have adopted secular philosophies (eg. humanism and skepticism), there is no one ideology or set of behaviors to which all atheists adhere.'))


soc.religion.christian


In [ ]:
print(predictLabel (10, 'President Dwight D. Eisenhower established NASA in 1958 with a distinctly civilian (rather than military) orientation encouraging peaceful applications in space science. The National Aeronautics and Space Act was passed on July 29, 1958, disestablishing NASAs predecessor, the National Advisory Committee for Aeronautics (NACA). The new agency became operational on October 1, 1958. Since that time, most US space exploration efforts have been led by NASA, including the Apollo moon-landing missions, the Skylab space station, and later the Space Shuttle. Currently, NASA is supporting the International Space Station and is overseeing the development of the Orion Multi-Purpose Crew Vehicle, the Space Launch System and Commercial Crew vehicles. The agency is also responsible for the Launch Services Program (LSP) which provides oversight of launch operations and countdown management for unmanned NASA launches.'))


sci.space


In [ ]:
print(predictLabel (10, 'The transistor is the fundamental building block of modern electronic devices, and is ubiquitous in modern electronic systems. First conceived by Julius Lilienfeld in 1926 and practically implemented in 1947 by American physicists John Bardeen, Walter Brattain, and William Shockley, the transistor revolutionized the field of electronics, and paved the way for smaller and cheaper radios, calculators, and computers, among other things. The transistor is on the list of IEEE milestones in electronics, and Bardeen, Brattain, and Shockley shared the 1956 Nobel Prize in Physics for their achievement.'))


talk.politics.mideast


In [ ]:
print(predictLabel (10, 'The Colt Single Action Army which is also known as the Single Action Army, SAA, Model P, Peacemaker, M1873, and Colt .45 is a single-action revolver with a revolving cylinder holding six metallic cartridges. It was designed for the U.S. government service revolver trials of 1872 by Colts Patent Firearms Manufacturing Company – todays Colts Manufacturing Company – and was adopted as the standard military service revolver until 1892. The Colt SAA has been offered in over 30 different calibers and various barrel lengths. Its overall appearance has remained consistent since 1873. Colt has discontinued its production twice, but brought it back due to popular demand. The revolver was popular with ranchers, lawmen, and outlaws alike, but as of the early 21st century, models are mostly bought by collectors and re-enactors. Its design has influenced the production of numerous other models from other companies.'))


sci.crypt


In [ ]:
print(predictLabel (10, 'Howe was recruited by the Red Wings and made his NHL debut in 1946. He led the league in scoring each year from 1950 to 1954, then again in 1957 and 1963. He ranked among the top ten in league scoring for 21 consecutive years and set a league record for points in a season (95) in 1953. He won the Stanley Cup with the Red Wings four times, won six Hart Trophies as the leagues most valuable player, and won six Art Ross Trophies as the leading scorer. Howe retired in 1971 and was inducted into the Hockey Hall of Fame the next year. However, he came back two years later to join his sons Mark and Marty on the Houston Aeros of the WHA. Although in his mid-40s, he scored over 100 points twice in six years. He made a brief return to the NHL in 1979–80, playing one season with the Hartford Whalers, then retired at the age of 52. His involvement with the WHA was central to their brief pre-NHL merger success and forced the NHL to expand their recruitment to European talent and to expand to new markets.'))


rec.sport.baseball


## Stop Spark context



In [ ]:
sc.stop()

## run on the entire dataset in EMR cluster

In [ ]:
# Task 1
import re
import numpy as np

numWords = 20000
corpus = sc.textFile ("s3://risamyersbucket/Lab/text/20_news_same_line.txt")
validLines = corpus.filter(lambda x : 'id=' in x)
keyAndText = validLines.map(lambda x : (x[x.index('id="') + 4 : x.index('" url=')], x[x.index('"> ') + 3:x.index(' </doc>')]))
regex = re.compile('[^a-zA-Z]')
keyAndListOfWords = keyAndText.map(lambda x : (str(x[0]), regex.sub(' ', x[1]).lower().split()))
allWords = keyAndListOfWords.flatMap(lambda x: ((j, 1) for j in x[1]))
allCounts = allWords.reduceByKey (lambda a, b: a + b)
topWords = allCounts.top (numWords, lambda x : x[1])
numWordsRDD = sc.parallelize(range(numWords))
refDict = numWordsRDD.map(lambda x:(topWords[x][0],x))
RDD1= keyAndListOfWords.flatMap(lambda x: ((i, x[0])for i in x[1]))
RDD2= RDD1.join(refDict)
RDD3= RDD2.map(lambda x: ((x[1])))
zeros= np.zeros(20000)
def seq_func(accumulator, element):
  accumulator[element]=accumulator[element]+1
  return accumulator

def comb_func(accumulator1, accumulator2):
  sum= accumulator1+accumulator2
  return sum

bag_of_words = RDD3.aggregateByKey (zeros,seq_func, comb_func)
arr1_1 = np.array(bag_of_words.lookup("20_newsgroups/soc.religion.christian/21626"))
print("20_newsgroups/soc.religion.christian/21626 ... \n", arr1_1[arr1_1.nonzero()])
arr1_2 = np.array(bag_of_words.lookup("20_newsgroups/talk.politics.misc/179019"))
print("20_newsgroups/talk.politics.misc/179019 ...\n", arr1_2[arr1_2.nonzero()])
arr1_3 = np.array(bag_of_words.lookup("20_newsgroups/rec.autos/103167"))
print("20_newsgroups/rec.autos/103167 ... \n", arr1_3[arr1_3.nonzero()])

# Task 2
np.set_printoptions(suppress=True)
rankAndid = RDD2.map(lambda x: (x[1][1], x[1][0]))
rankAndidcnt = rankAndid.groupByKey()
rankAndocc = rankAndidcnt.map(lambda x: (x[0], len(set(x[1]))))
rankAndoccSorted = rankAndocc.sortBy(lambda x: x[0])
idf = rankAndoccSorted.map(lambda x: (np.log(19997.0/x[1]))).collect()
tfidf = bag_of_words.map(lambda x: (x[0], np.multiply((x[1]/x[1].sum()),idf)))
arr2_1 = np.array(tfidf.lookup('20_newsgroups/soc.religion.christian/21626'))
print('20_newsgroups/soc.religion.christian/21626 ... \n', arr2_1[arr2_1.nonzero()])
arr2_2 = np.array(tfidf.lookup("20_newsgroups/talk.politics.misc/179019"))
print("20_newsgroups/talk.politics.misc/179019 ...\n", arr2_2[arr2_2.nonzero()])
arr2_3 = np.array(tfidf.lookup("20_newsgroups/rec.autos/103167"))
print("20_newsgroups/rec.autos/103167 ... \n", arr2_3[arr2_3.nonzero()])

# Task 3
def predictLabel (k, test_doc):
    regex = re.compile('[^a-zA-Z]')
    words = sc.parallelize(regex.sub(' ', test_doc).lower().split())
    allWordsCnt = words.map(lambda x: (x,1)).reduceByKey(lambda a,b:a+b)
    allWordsCntandRank = allWordsCnt.join(refDict)

    rankandCnt = allWordsCntandRank.map(lambda x: (x[1][1], x[1][0]))

    countvec = np.zeros(20000)
    for pair in rankandCnt.collect():
      countvec[pair[0]] = pair[1]

    tf_idf_test = np.multiply(countvec/countvec.sum(), idf)

    l2_dist = tfidf.map(lambda x: (x[0], np.linalg.norm(x[1]-tf_idf_test)))

    knn_candidate = l2_dist.top(k, key = lambda x: -x[1])
    groupnames = [(n[0].split('/')[1], n[1]) for n in knn_candidate]
    category_counts = {}
    for cat, dist in groupnames:
      if cat in category_counts:
        category_counts[cat] += 1
      else:
        category_counts[cat] = 1
    max_count = max(category_counts.values())
    most_common_categories = [category for category, count in category_counts.items() if count == max_count]

    if len(most_common_categories) == 1:
      return most_common_categories[0]
    else:
      min_dist = float('inf')
      min_category = None
      for category, dist in groupnames:
          if category in most_common_categories and dist < min_dist:
            min_dist = dist
            min_category = category
      return min_category

predictions = []
predictions.append(predictLabel (10, 'Graphics are pictures and movies created using computers – usually referring to image data created by a computer specifically with help from specialized graphical hardware and software. It is a vast and recent area in computer science. The phrase was coined by computer graphics researchers Verne Hudson and William Fetter of Boeing in 1960. It is often abbreviated as CG, though sometimes erroneously referred to as CGI. Important topics in computer graphics include user interface design, sprite graphics, vector graphics, 3D modeling, shaders, GPU design, implicit surface visualization with ray tracing, and computer vision, among others. The overall methodology depends heavily on the underlying sciences of geometry, optics, and physics. Computer graphics is responsible for displaying art and image data effectively and meaningfully to the user, and processing image data received from the physical world. The interaction and understanding of computers and interpretation of data has been made easier because of computer graphics. Computer graphic development has had a significant impact on many types of media and has revolutionized animation, movies, advertising, video games, and graphic design generally.'))
predictions.append(predictLabel (10, 'A deity is a concept conceived in diverse ways in various cultures, typically as a natural or supernatural being considered divine or sacred. Monotheistic religions accept only one Deity (predominantly referred to as God), polytheistic religions accept and worship multiple deities, henotheistic religions accept one supreme deity without denying other deities considering them as equivalent aspects of the same divine principle, while several non-theistic religions deny any supreme eternal creator deity but accept a pantheon of deities which live, die and are reborn just like any other being. A male deity is a god, while a female deity is a goddess. The Oxford reference defines deity as a god or goddess (in a polytheistic religion), or anything revered as divine. C. Scott Littleton defines a deity as a being with powers greater than those of ordinary humans, but who interacts with humans, positively or negatively, in ways that carry humans to new levels of consciousness beyond the grounded preoccupations of ordinary life.'))
predictions.append(predictLabel (10, 'Egypt, officially the Arab Republic of Egypt, is a transcontinental country spanning the northeast corner of Africa and southwest corner of Asia by a land bridge formed by the Sinai Peninsula. Egypt is a Mediterranean country bordered by the Gaza Strip and Israel to the northeast, the Gulf of Aqaba to the east, the Red Sea to the east and south, Sudan to the south, and Libya to the west. Across the Gulf of Aqaba lies Jordan, and across from the Sinai Peninsula lies Saudi Arabia, although Jordan and Saudi Arabia do not share a land border with Egypt. It is the worlds only contiguous Eurafrasian nation. Egypt has among the longest histories of any modern country, emerging as one of the worlds first nation states in the tenth millennium BC. Considered a cradle of civilisation, Ancient Egypt experienced some of the earliest developments of writing, agriculture, urbanisation, organised religion and central government. Iconic monuments such as the Giza Necropolis and its Great Sphinx, as well the ruins of Memphis, Thebes, Karnak, and the Valley of the Kings, reflect this legacy and remain a significant focus of archaeological study and popular interest worldwide. Egypts rich cultural heritage is an integral part of its national identity, which has endured, and at times assimilated, various foreign influences, including Greek, Persian, Roman, Arab, Ottoman, and European. One of the earliest centers of Christianity, Egypt was Islamised in the seventh century and remains a predominantly Muslim country, albeit with a significant Christian minority.'))
predictions.append(predictLabel (10, 'The term atheism originated from the Greek atheos, meaning without god(s), used as a pejorative term applied to those thought to reject the gods worshiped by the larger society. With the spread of freethought, skeptical inquiry, and subsequent increase in criticism of religion, application of the term narrowed in scope. The first individuals to identify themselves using the word atheist lived in the 18th century during the Age of Enlightenment. The French Revolution, noted for its unprecedented atheism, witnessed the first major political movement in history to advocate for the supremacy of human reason. Arguments for atheism range from the philosophical to social and historical approaches. Rationales for not believing in deities include arguments that there is a lack of empirical evidence; the problem of evil; the argument from inconsistent revelations; the rejection of concepts that cannot be falsified; and the argument from nonbelief. Although some atheists have adopted secular philosophies (eg. humanism and skepticism), there is no one ideology or set of behaviors to which all atheists adhere.'))
predictions.append(predictLabel (10, 'President Dwight D. Eisenhower established NASA in 1958 with a distinctly civilian (rather than military) orientation encouraging peaceful applications in space science. The National Aeronautics and Space Act was passed on July 29, 1958, disestablishing NASAs predecessor, the National Advisory Committee for Aeronautics (NACA). The new agency became operational on October 1, 1958. Since that time, most US space exploration efforts have been led by NASA, including the Apollo moon-landing missions, the Skylab space station, and later the Space Shuttle. Currently, NASA is supporting the International Space Station and is overseeing the development of the Orion Multi-Purpose Crew Vehicle, the Space Launch System and Commercial Crew vehicles. The agency is also responsible for the Launch Services Program (LSP) which provides oversight of launch operations and countdown management for unmanned NASA launches.'))
predictions.append(predictLabel (10, 'The transistor is the fundamental building block of modern electronic devices, and is ubiquitous in modern electronic systems. First conceived by Julius Lilienfeld in 1926 and practically implemented in 1947 by American physicists John Bardeen, Walter Brattain, and William Shockley, the transistor revolutionized the field of electronics, and paved the way for smaller and cheaper radios, calculators, and computers, among other things. The transistor is on the list of IEEE milestones in electronics, and Bardeen, Brattain, and Shockley shared the 1956 Nobel Prize in Physics for their achievement.'))
predictions.append(predictLabel (10, 'The Colt Single Action Army which is also known as the Single Action Army, SAA, Model P, Peacemaker, M1873, and Colt .45 is a single-action revolver with a revolving cylinder holding six metallic cartridges. It was designed for the U.S. government service revolver trials of 1872 by Colts Patent Firearms Manufacturing Company – todays Colts Manufacturing Company – and was adopted as the standard military service revolver until 1892. The Colt SAA has been offered in over 30 different calibers and various barrel lengths. Its overall appearance has remained consistent since 1873. Colt has discontinued its production twice, but brought it back due to popular demand. The revolver was popular with ranchers, lawmen, and outlaws alike, but as of the early 21st century, models are mostly bought by collectors and re-enactors. Its design has influenced the production of numerous other models from other companies.'))
predictions.append(predictLabel (10, 'Howe was recruited by the Red Wings and made his NHL debut in 1946. He led the league in scoring each year from 1950 to 1954, then again in 1957 and 1963. He ranked among the top ten in league scoring for 21 consecutive years and set a league record for points in a season (95) in 1953. He won the Stanley Cup with the Red Wings four times, won six Hart Trophies as the leagues most valuable player, and won six Art Ross Trophies as the leading scorer. Howe retired in 1971 and was inducted into the Hockey Hall of Fame the next year. However, he came back two years later to join his sons Mark and Marty on the Houston Aeros of the WHA. Although in his mid-40s, he scored over 100 points twice in six years. He made a brief return to the NHL in 1979–80, playing one season with the Hartford Whalers, then retired at the age of 52. His involvement with the WHA was central to their brief pre-NHL merger success and forced the NHL to expand their recruitment to European talent and to expand to new markets.'))
print(predictions)